In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
ds = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")
def get_instruct_response(item):
    # item["instruction"] = item['chosen'][0]["content"]
    item["response"] = item['chosen'][1]["content"]
    return item
ds = ds.map(get_instruct_response, batch_size=1024, num_proc=8)
export_data = ds.select_columns(["prompt", "response"])
export_data

/home/lidong1/miniconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['prompt', 'response'],
    num_rows: 61135
})

In [5]:
response_1 = export_data['response'][0]

In [16]:
from selenium import webdriver  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options  
from PIL import Image  
import os  
import markdown  
import logging  
import time
from io import BytesIO  

# 配置日志  
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')  
  
def markdown_to_html(markdown_text):  
    """Convert Markdown text to HTML"""  
    html = markdown.markdown(markdown_text)  
    return html  
  
def save_html_to_file(html_content, file_path):  
    """Save HTML content to a file"""  
    with open(file_path, 'w', encoding='utf-8') as file:  
        file.write(html_content)  

def html_to_image(html_file, output_image):  
    # Set Chrome options  
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  # Headless mode  
    chrome_options.add_argument("--disable-gpu")  
    chrome_options.add_argument("--no-sandbox")  
      
    # Set ChromeDriver service  
    service = Service('/usr/local/bin/chromedriver')  # Path to ChromeDriver  
      
    # Start Chrome browser  
    driver = webdriver.Chrome(service=service, options=chrome_options)  
      
    # Open HTML file  
    file_url = f"file://{os.path.abspath(html_file)}"  
    driver.get(file_url)  
    time.sleep(2)  
      
    # Get the dimensions of the page  
    total_width = driver.execute_script("return document.body.scrollWidth")  
    total_height = driver.execute_script("return document.body.scrollHeight")  
    viewport_height = driver.execute_script("return window.innerHeight")  
      
    rectangles = []  
      
    # Split the screen in multiple parts  
    for i in range(0, total_height, viewport_height):  
        driver.execute_script(f"window.scrollTo(0, {i});")  
        time.sleep(0.5)  
        screenshot = driver.get_screenshot_as_png()  
        screenshot = Image.open(BytesIO(screenshot))  
        rectangles.append(screenshot)  
      
    # Combine images  
    combined_image = Image.new('RGB', (total_width, total_height))  
    for i, image in enumerate(rectangles):  
        combined_image.paste(image, (0, i * viewport_height))  
      
    # Save the final image  
    combined_image.save(output_image)  
      
    # Close browser  
    driver.quit()  

# def html_to_image(html_file, output_image):  
#     # Set Chrome options  
#     chrome_options = Options()  
#     chrome_options.add_argument("--headless")  # Headless mode  
#     chrome_options.add_argument("--disable-gpu")  
#     chrome_options.add_argument("--no-sandbox")  
      
#     # Set ChromeDriver service  
#     service = Service('/usr/local/bin/chromedriver')  # Path to ChromeDriver  
      
#     # Start Chrome browser  
#     driver = webdriver.Chrome(service=service, options=chrome_options)  
      
#     # Open HTML file  
#     file_url = f"file://{os.path.abspath(html_file)}"  
#     driver.get(file_url)  
#     time.sleep(2)  
      
#     # Get the dimensions of the page  
#     total_width = driver.execute_script("return document.body.scrollWidth")  
#     total_height = driver.execute_script("return document.body.scrollHeight")  
      
#     # Set the window size to the dimensions of the page  
#     driver.set_window_size(total_width, total_height)  
#     time.sleep(2)  
      
#     # Take screenshot  
#     screenshot = driver.get_screenshot_as_png()  
      
#     # Save image  
#     with open(output_image, 'wb') as file:  
#         file.write(screenshot)  
      
#     # Close browser  
#     driver.quit()  
      
#     # Use PIL to adjust image size  
#     with Image.open(output_image) as img:  
#         img = img.crop(img.getbbox())  # Crop blank areas  
#         img.save(output_image)  
  
  
  
def text_to_image(text, output_image):  
    """Convert text to image and log the process"""  
    logging.info("Starting the conversion process.")  
      
    # Convert text to Markdown (assume the text is in Markdown format)  
    html_content = markdown_to_html(text)  
      
    # Wrap HTML header and style  
    html_content = f"""  
    <!DOCTYPE html>  
    <html lang="en">  
    <head>  
        <meta charset="UTF-8">  
        <title>Markdown to HTML Example</title>  
        <style>  
            body {{  
                font-family: 'Arial', sans-serif;  
            }}  
        </style>  
    </head>  
    <body>  
        {html_content}  
    </body>  
    </html>  
    """  
      
    # Get the directory of the current script  
    # script_dir = os.path.dirname(os.path.abspath(__file__))  
    script_dir = './'
    # Save HTML to file  
    html_file = os.path.join(script_dir, 'example.html')  
    save_html_to_file(html_content, html_file)  
    logging.info(f"HTML content saved to {html_file}.")  
      
    # Convert HTML to image  
    output_image_path = os.path.join(script_dir, output_image)  
    html_to_image(html_file, output_image_path)  
    logging.info(f"Image saved to {output_image_path}.")  
      
    # Delete HTML file  
    os.remove(html_file)  
    logging.info(f"Temporary HTML file {html_file} deleted.")  
      
    logging.info("Conversion process completed successfully.")  
  
markdown_text = """  
# Hello, World!  

This is an example document written using **Markdown**.  

- Item 1  
- Item 2  
- Item 3  

[Click here](http://example.com) to visit the example website.  
""" 
# if __name__ == "__main__":
#     # 示例Markdown文本  
  
    
#     # 调用函数并输出日志  
#     output_image = 'output.png'  
#     text_to_image(markdown_text, output_image)  


In [25]:
from html2image import Html2Image
hti = Html2Image(temp_path="/home/lidong1/jianglingjie/temp")
hti.screenshot(url='https://www.python.org', save_as='python_org.png')

[0823/004648.007068:WARNING:sandbox_linux.cc(430)] InitializeSandbox() called with multiple threads in process gpu-process.
[0823/004648.012217:WARNING:bluez_dbus_manager.cc(248)] Floss manager not present, cannot set Floss enable/disable.
173264 bytes written to file /home/lidong1/jianglingjie/LLama-Factory/data_process/python_org.png


['/home/lidong1/jianglingjie/LLama-Factory/data_process/python_org.png']

In [26]:
html = """<h1> An interesting title </h1> This page will be red"""
css = "body {background: red;}"

hti.screenshot(html_str=html, css_str=css, save_as='red_page.png')

[0823/004655.789946:WARNING:bluez_dbus_manager.cc(248)] Floss manager not present, cannot set Floss enable/disable.
[0823/004655.791542:WARNING:sandbox_linux.cc(430)] InitializeSandbox() called with multiple threads in process gpu-process.
13187 bytes written to file /home/lidong1/jianglingjie/LLama-Factory/data_process/red_page.png


['/home/lidong1/jianglingjie/LLama-Factory/data_process/red_page.png']

In [19]:
with open("response.txt", 'w') as f:
    f.write(response_1)

In [18]:
from text2img import text_to_image
output_path = "text_images/output.png"
text = response_1
text_to_image(text, output_path)

2024-08-23 00:15:53,993 - INFO - Starting the conversion process.
2024-08-23 00:15:53,998 - INFO - HTML content saved to /home/lidong1/jianglingjie/LLama-Factory/data_process/example.html.
2024-08-23 00:15:56,561 - INFO - Image saved to /home/lidong1/jianglingjie/LLama-Factory/data_process/text_images/output.png.
2024-08-23 00:15:56,562 - INFO - Temporary HTML file /home/lidong1/jianglingjie/LLama-Factory/data_process/example.html deleted.
2024-08-23 00:15:56,563 - INFO - Conversion process completed successfully.


In [3]:
def get_data():
    ds = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")
    def get_instruct_response(item):
        # item["instruction"] = item['chosen'][0]["content"]
        item["response"] = item['chosen'][1]["content"]
        return item
    ds = ds.map(get_instruct_response, batch_size=1024, num_proc=8)
    export_data = ds.select_columns(["prompt", "response"])
    return export_data

In [4]:
response = export_data['response'][0]

In [22]:
# # import openai  
# from PIL import Image, ImageDraw, ImageFont  
# import textwrap  
  
# # 设置OpenAI API密钥  
# # openai.api_key = 'your_openai_api_key'  
  
# # # 调用GPT接口生成文本  
# # response = openai.Completion.create(  
# #     engine="text-davinci-003",  
# #     prompt="写一段关于人工智能的短文。",  
# #     max_tokens=150  
# # )  
# response = export_data['response'][0]
# generated_text = response
  
# # 创建一个空白图片  
# width, height = 800, 600  
# image = Image.new('RGB', (width, height), color=(255, 255, 255))  
  
# # 设置字体和大小  
# try:  
#     font = ImageFont.truetype("arial.ttf", 20)  
# except IOError:  
#     font = ImageFont.load_default()  
  
# # 创建绘图对象  
# draw = ImageDraw.Draw(image)  
  
# # 设置文本换行  
# margin = 40  
# offset = 40  
# for line in textwrap.wrap(generated_text, width=70):  
#     draw.text((margin, offset), line, font=font, fill=(0, 0, 0))  
#     offset += font.getsize(line)[1] + 5  
  
# # 保存图片  
# image.save('generated_text_image.png')  
# print("图片已保存为 'generated_text_image.png'")

AttributeError: 'FreeTypeFont' object has no attribute 'getsize'

In [5]:
import markdown  
from html2image import Html2Image  
  
# Markdown 文本  
markdown_text = """  
# 这是一个标题  
  
这是一些文本。  
  
- 列表项 1  
- 列表项 2  
- 列表项 3  
"""  
  
# 将 Markdown 转换为 HTML  
html = markdown.markdown(markdown_text)  
print(html)
# 创建 Html2Image 对象  
hti = Html2Image()  
  
# 将 HTML 转换为图片  
hti.screenshot(html_str=html, save_as='output.png')  
  
print("图片已生成并保存为 output.png")  


<h1>这是一个标题</h1>
<p>这是一些文本。  </p>
<ul>
<li>列表项 1  </li>
<li>列表项 2  </li>
<li>列表项 3  </li>
</ul>


[0822/201007.264298:WARNING:bluez_dbus_manager.cc(248)] Floss manager not present, cannot set Floss enable/disable.
[0822/201007.268170:WARNING:sandbox_linux.cc(430)] InitializeSandbox() called with multiple threads in process gpu-process.


图片已生成并保存为 output.png


8143 bytes written to file /home/lidong1/jianglingjie/LLama-Factory/data_process/output.png


In [12]:
import imgkit  
# Markdown 文本  
markdown_text = """  
# 这是一个标题  
  
这是一些文本。  
  
- 列表项 1  
- 列表项 2  
- 列表项 3  
"""  
  
# 将 Markdown 转换为 HTML  
html_string = markdown.markdown(markdown_text)    
# html_string = '<html><body><h1>Hello World</h1></body></html>'  
with open("temp.html", 'w', encoding='utf-8') as f:
    f.write(html_string)
    
output_image = 'output_from_string.png'  
options = {  
    'encoding': "UTF-8",  
    'custom-header': [  
        ('Accept-Encoding', 'gzip')  
    ],  
    'quiet': ''  
}  
imgkit.from_file("temp.html", output_image, options=options)  


True

In [14]:
import markdown  
import imgkit  
  
def markdown_to_html(markdown_text):  
    """将Markdown文本转换为HTML"""  
    html = markdown.markdown(markdown_text)  
    return html  
  
def save_html_to_file(html_content, file_path):  
    """将HTML内容保存到文件"""  
    with open(file_path, 'w', encoding='utf-8') as file:  
        file.write(html_content)  
  
def html_to_image(html_file, output_image):  
    """将HTML文件转换为图片"""  
    options = {  
        'encoding': 'UTF-8',  
        'custom-header': [  
            ('Accept-Encoding', 'gzip')  
        ],  
        'quiet': ''  
    }  
    imgkit.from_file(html_file, output_image, options=options)  
  
# 示例Markdown文本  
markdown_text = """  
# 你好，世界！  
  
这是一个使用 **Markdown** 编写的示例文档。  
  
- 项目1  
- 项目2  
- 项目3  
  
[点击这里](http://example.com) 访问示例网站。  
"""  
  
# 将Markdown转换为HTML  
html_content = markdown_to_html(markdown_text)  
  
# 包装HTML头部和样式  
html_content = f"""  
<!DOCTYPE html>  
<html lang="zh-CN">  
<head>  
    <meta charset="UTF-8">  
    <title>Markdown 转 HTML 示例</title>  
    <style>  
        body {{  
            font-family: 'WenQuanYi Zen Hei', 'Arial', sans-serif;  
        }}  
    </style>  
</head>  
<body>  
    {html_content}  
</body>  
</html>  
"""  
  
# 保存HTML到文件  
html_file = 'example.html'  
save_html_to_file(html_content, html_file)  
  
# 将HTML转换为图片  
output_image = 'output.png'  
html_to_image(html_file, output_image)  


In [2]:
from selenium import webdriver  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options  
from PIL import Image  
import markdown  
import os  

def markdown_to_html(markdown_text):  
    """将Markdown文本转换为HTML"""  
    html = markdown.markdown(markdown_text)  
    return html  
  
def save_html_to_file(html_content, file_path):  
    """将HTML内容保存到文件"""  
    with open(file_path, 'w', encoding='utf-8') as file:  
        file.write(html_content)  

def html_to_image(html_file, output_image):  
    # 设置Chrome选项  
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  # 无头模式  
    chrome_options.add_argument("--disable-gpu")  
    chrome_options.add_argument("--window-size=1920x1080")  
    chrome_options.add_argument("--no-sandbox")  
      
    # 设置ChromeDriver服务  
    service = Service('/usr/local/bin/chromedriver')  # ChromeDriver的路径  
      
    # 启动Chrome浏览器  
    driver = webdriver.Chrome(service=service, options=chrome_options)  
      
    # 打开HTML文件  
    file_url = f"file://{os.path.abspath(html_file)}"  
    driver.get(file_url)  
      
    # 截图  
    screenshot = driver.get_screenshot_as_png()  
      
    # 保存图片  
    with open(output_image, 'wb') as file:  
        file.write(screenshot)  
      
    # 关闭浏览器  
    driver.quit()  
  
    # 使用PIL调整图片大小  
    with Image.open(output_image) as img:  
        img = img.crop(img.getbbox())  # 剪裁空白区域  
        img.save(output_image)  

# 示例Markdown文本  
markdown_text = """  
# 你好，世界！  
  
这是一个使用 **Markdown** 编写的示例文档。  
  
- 项目1  
- 项目2  
- 项目3  
  
[点击这里](http://example.com) 访问示例网站。  
"""  
  
# 将Markdown转换为HTML  
html_content = markdown_to_html(markdown_text)  
  
# 包装HTML头部和样式  
html_content = f"""  
<!DOCTYPE html>  
<html lang="zh-CN">  
<head>  
    <meta charset="UTF-8">  
    <title>Markdown 转 HTML 示例</title>  
    <style>  
        body {{  
            font-family: 'WenQuanYi Zen Hei', 'Arial', sans-serif;  
        }}  
    </style>  
</head>  
<body>  
    {html_content}  
</body>  
</html>  
"""  
  
# 保存HTML到文件  
html_file = 'example.html'  
save_html_to_file(html_content, html_file)  
  
# 将HTML转换为图片  
output_image = 'output.png'  
html_to_image(html_file, output_image)    

# # 使用示例  
# html_file = 'example.html'  
# output_image = 'output.png'  
html_to_image(html_file, output_image)  


In [1]:
from selenium import webdriver  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options  
from PIL import Image  

import os  

def markdown_to_html(markdown_text):  
    """将Markdown文本转换为HTML"""  
    html = markdown.markdown(markdown_text)  
    return html  
  
def save_html_to_file(html_content, file_path):  
    """将HTML内容保存到文件"""  
    with open(file_path, 'w', encoding='utf-8') as file:  
        file.write(html_content)  

def html_to_image(html_file, output_image):  
    # 设置Chrome选项  
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  # 无头模式  
    chrome_options.add_argument("--disable-gpu")  
    chrome_options.add_argument("--window-size=1920x1080")  
    chrome_options.add_argument("--no-sandbox")  
      
    # 设置ChromeDriver服务  
    service = Service('/usr/local/bin/chromedriver')  # ChromeDriver的路径  
      
    # 启动Chrome浏览器  
    driver = webdriver.Chrome(service=service, options=chrome_options)  
      
    # 打开HTML文件  
    file_url = f"file://{os.path.abspath(html_file)}"  
    driver.get(file_url)  
      
    # 截图  
    screenshot = driver.get_screenshot_as_png()  
      
    # 保存图片  
    with open(output_image, 'wb') as file:  
        file.write(screenshot)  
      
    # 关闭浏览器  
    driver.quit()  
  
    # 使用PIL调整图片大小  
    with Image.open(output_image) as img:  
        img = img.crop(img.getbbox())  # 剪裁空白区域  
        img.save(output_image)  

# 示例Markdown文本  
markdown_text = """  
# 你好，世界！  
  
这是一个使用 **Markdown** 编写的示例文档。  
  
- 项目1  
- 项目2  
- 项目3  
  
[点击这里](http://example.com) 访问示例网站。  
"""  
  
# 将Markdown转换为HTML  
html_content = markdown_to_html(markdown_text)  
  
# 包装HTML头部和样式  
html_content = f"""  
<!DOCTYPE html>  
<html lang="zh-CN">  
<head>  
    <meta charset="UTF-8">  
    <title>Markdown 转 HTML 示例</title>  
    <style>  
        body {{  
            font-family: 'WenQuanYi Zen Hei', 'Arial', sans-serif;  
        }}  
    </style>  
</head>  
<body>  
    {html_content}  
</body>  
</html>  
"""  
  
# 保存HTML到文件  
html_file = 'example.html'  
save_html_to_file(html_content, html_file)  
  
# 将HTML转换为图片  
output_image = 'output.png'  
html_to_image(html_file, output_image)    

# # 使用示例  
# html_file = 'example.html'  
# output_image = 'output.png'  
html_to_image(html_file, output_image)  


NameError: name 'markdown' is not defined

In [5]:
from selenium import webdriver  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options  
from PIL import Image  
import os  
import markdown  
import logging  
  
# 配置日志  
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')  
  
def markdown_to_html(markdown_text):  
    """Convert Markdown text to HTML"""  
    html = markdown.markdown(markdown_text)  
    return html  
  
def save_html_to_file(html_content, file_path):  
    """Save HTML content to a file"""  
    with open(file_path, 'w', encoding='utf-8') as file:  
        file.write(html_content)  
  
def html_to_image(html_file, output_image):  
    # Set Chrome options  
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  # Headless mode  
    chrome_options.add_argument("--disable-gpu")  
    chrome_options.add_argument("--window-size=1920x1080")  
    chrome_options.add_argument("--no-sandbox")  
      
    # Set ChromeDriver service  
    service = Service('/usr/local/bin/chromedriver')  # Path to ChromeDriver  
      
    # Start Chrome browser  
    driver = webdriver.Chrome(service=service, options=chrome_options)  
      
    # Open HTML file  
    file_url = f"file://{os.path.abspath(html_file)}"  
    driver.get(file_url)  
      
    # Take screenshot  
    screenshot = driver.get_screenshot_as_png()  
      
    # Save image  
    with open(output_image, 'wb') as file:  
        file.write(screenshot)  
      
    # Close browser  
    driver.quit()  
      
    # Use PIL to adjust image size  
    with Image.open(output_image) as img:  
        img = img.crop(img.getbbox())  # Crop blank areas  
        img.save(output_image)  
  
def text_to_image_with_log(text, output_image):  
    """  
    Convert text to image and log the process  
    """  
    logging.info("Starting the conversion process.")  
      
    # Convert text to Markdown (assume the text is in Markdown format)  
    html_content = markdown_to_html(text)  
      
    # Wrap HTML header and style  
    html_content = f"""  
    <!DOCTYPE html>  
    <html lang="en">  
    <head>  
        <meta charset="UTF-8">  
        <title>Markdown to HTML Example</title>  
        <style>  
            body {{  
                font-family: 'Arial', sans-serif;  
            }}  
        </style>  
    </head>  
    <body>  
        {html_content}  
    </body>  
    </html>  
    """  
      
    # Save HTML to file  
    html_file = 'example.html'  
    save_html_to_file(html_content, html_file)  
    # logging.info(f"HTML content saved to {html_file}.")  
      
    # Convert HTML to image  
    html_to_image(html_file, output_image)  
    logging.info(f"Image saved to {output_image}.")  
      
    # Delete HTML file  
    os.remove(html_file)  
    # logging.info(f"Temporary HTML file {html_file} deleted.")  
      
    logging.info("Conversion process completed successfully.")  
  
# 示例Markdown文本  
markdown_text = """  
# Hello, World!  
  
This is an example document written using **Markdown**.  
  
- Item 1  
- Item 2  
- Item 3  
  
[Click here](http://example.com) to visit the example website.  
"""  
  
# 调用函数并输出日志  
output_image = 'output.png'  
text_to_image_with_log(markdown_text, output_image)  


2024-08-22 21:55:36,539 - INFO - Starting the conversion process.
2024-08-22 21:55:37,117 - INFO - Image saved to output.png.
2024-08-22 21:55:37,118 - INFO - Conversion process completed successfully.


In [6]:
from selenium import webdriver  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options  
from PIL import Image  
import os  
import markdown  
import logging  
  
# 配置日志  
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')  
  
def markdown_to_html(markdown_text):  
    """Convert Markdown text to HTML"""  
    html = markdown.markdown(markdown_text)  
    return html  
  
def save_html_to_file(html_content, file_path):  
    """Save HTML content to a file"""  
    with open(file_path, 'w', encoding='utf-8') as file:  
        file.write(html_content)  
  
def html_to_image(html_file, output_image):  
    # Set Chrome options  
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  # Headless mode  
    chrome_options.add_argument("--disable-gpu")  
    chrome_options.add_argument("--window-size=1920x1080")  
    chrome_options.add_argument("--no-sandbox")  
      
    # Set ChromeDriver service  
    service = Service('/usr/local/bin/chromedriver')  # Path to ChromeDriver  
      
    # Start Chrome browser  
    driver = webdriver.Chrome(service=service, options=chrome_options)  
      
    # Open HTML file  
    file_url = f"file://{os.path.abspath(html_file)}"  
    driver.get(file_url)  
      
    # Take screenshot  
    screenshot = driver.get_screenshot_as_png()  
      
    # Save image  
    with open(output_image, 'wb') as file:  
        file.write(screenshot)  
      
    # Close browser  
    driver.quit()  
      
    # Use PIL to adjust image size  
    with Image.open(output_image) as img:  
        img = img.crop(img.getbbox())  # Crop blank areas  
        img.save(output_image)  
  
def text_to_image_with_log(text, output_image):  
    """  
    Convert text to image and log the process  
    """  
    logging.info("Starting the conversion process.")  
      
    # Convert text to Markdown (assume the text is in Markdown format)  
    html_content = markdown_to_html(text)  
      
    # Wrap HTML header and style  
    html_content = f"""  
    <!DOCTYPE html>  
    <html lang="en">  
    <head>  
        <meta charset="UTF-8">  
        <title>Markdown to HTML Example</title>  
        <style>  
            body {{  
                font-family: 'Arial', sans-serif;  
            }}  
        </style>  
    </head>  
    <body>  
        {html_content}  
    </body>  
    </html>  
    """  
      
    # Save HTML to file  
    html_file = 'example.html'  
    save_html_to_file(html_content, html_file)  
    logging.info(f"HTML content saved to {html_file}.")  
      
    # Convert HTML to image  
    html_to_image(html_file, output_image)  
    logging.info(f"Image saved to {output_image}.")  
      
    # Delete HTML file  
    os.remove(html_file)  
    logging.info(f"Temporary HTML file {html_file} deleted.")  
      
    logging.info("Conversion process completed successfully.")  
  
# 示例Markdown文本  
markdown_text = """  
# Hello, World!  
  
This is an example document written using **Markdown**.  
  
- Item 1  
- Item 2  
- Item 3  
  
[Click here](http://example.com) to visit the example website.  
"""  
  
# 调用函数并输出日志  
output_image = 'output.png'  
text_to_image_with_log(markdown_text, output_image)  


2024-08-22 21:56:02,605 - INFO - Starting the conversion process.
2024-08-22 21:56:02,608 - INFO - HTML content saved to example.html.
2024-08-22 21:56:03,165 - INFO - Image saved to output.png.
2024-08-22 21:56:03,166 - INFO - Temporary HTML file example.html deleted.
2024-08-22 21:56:03,167 - INFO - Conversion process completed successfully.


In [3]:
def get_instruct_response(item):
    # item["instruction"] = item['chosen'][0]["content"]
    item["response"] = item['chosen'][1]["content"]
    return item
ds = ds.map(get_instruct_response, batch_size=1024, num_proc=8)
export_data = ds.select_columns(["prompt", "response"])
export_data

Dataset({
    features: ['prompt', 'response'],
    num_rows: 61135
})

In [5]:
from gpt4o import Openai, API_INFOS
client = Openai(apis=API_INFOS)
ans = client.get_response("你是gpt几？", system="you are gpt2.5, a helpful chat assisstant")
print(ans)

我是GPT-2.5，一个帮助用户回答问题和提供信息的聊天助手。有什么我可以帮你的吗？


我需要写一个prompt，让gpt帮我将LLM对于给定instruction生成的文本在文本美学方面进行修改的prompt，以下是我写的一个中文的system_template，对于这个template，如何让gpt更好的完成我的任务，给出修改意见和修改后的英语的system_template
system_template = \
"""请扮演一个文本改写者来改写LLM(Large Language Model)生成的文本，使其更具有文本美学。你的任务是对于提供的用户instruction和一个LLM生成的答案，你从文本美学的角度去思考这个答案是否合理，是否需要修改,然后给出修改的文本。
*Format*
文本美学分析:
[文本美学分析]
是否需要修改:[Y/N]
修改后的文本：
[修改后的文本]
"""

我需要写一个prompt，让gpt帮我将LLM对于给定instruction生成的文本在文本美学方面进行修改的prompt,主要从文本的排版和布局上对文本美观程度进行优化，例如段落长短、缩进、标题、列表、markdown形式的加粗、斜体、代码块，这些需要根据文本类型的需要来适当的修改，也可以添加一些其他方面的对文本美学的要求。帮我给出一些在观感上文本美学的构成要素，融入到prompt中。以下是我写的一个system template 和user template，对于这个template，如何让gpt更好的完成我的任务，给出修改意见和修改后的英语的template
system_template = \
"""You are tasked with acting as a text rewriter to enhance the aesthetic quality of text generated by a Large Language Model (LLM). Your job is to evaluate the provided user instruction and the LLM-generated response from an aesthetic perspective, determine whether the text needs modification, and then provide the revised text if necessary.   
    
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable]  
"""  
user_template = \
"""### User Instruction:
{instruction}

### Answer
{completition}

Your task is to:  
1. Analyze the LLM-generated response from a textual aesthetic perspective.  
2. Determine whether the text needs modification.  
3. Provide a revised version of the text if necessary.  
   
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable] 
"""

In [4]:
system_template = \
"""You are tasked with acting as a text rewriter to enhance the aesthetic quality of text generated by a Large Language Model (LLM). Your job is to evaluate the provided user instruction and the LLM-generated response from an aesthetic perspective, determine whether the text needs modification, and then provide the revised text if necessary.   
  
Consider the following aspects of textual aesthetics:  
- **Fluency**: Is the text smooth and easy to read?  
- **Vocabulary**: Are the words used varied and appropriate?  
- **Sentence Structure**: Are the sentences well-constructed and rhythmically balanced?  
- **Imagery and Tone**: Does the text evoke vivid images and maintain a consistent tone?  
  
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable]  
"""  

In [5]:
user_template = \
"""### User Instruction:
{instruction}

### Answer
{completition}

Your task is to:  
1. Analyze the LLM-generated response from a textual aesthetic perspective.  
2. Determine whether the text needs modification.  
3. Provide a revised version of the text if necessary.  
  
Consider the following aspects of textual aesthetics:  
- **Fluency**: Is the text smooth and easy to read?  
- **Vocabulary**: Are the words used varied and appropriate?  
- **Sentence Structure**: Are the sentences well-constructed and rhythmically balanced?  
- **Imagery and Tone**: Does the text evoke vivid images and maintain a consistent tone?  
  
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable] 
"""

In [6]:
system_template= \
"""You are tasked with acting as a text rewriter to enhance the aesthetic quality of text generated by a Large Language Model (LLM). Your job is to evaluate the provided user instruction and the LLM-generated response from an aesthetic perspective, determine whether the text needs modification, and then provide the revised text if necessary.  
  
*Format*   
  
Textual Aesthetic Analysis:
[Your analysis]

Does it need modification:
[Y/N]

Revised Text:
[Your revised text, if applicable]   
  
*Textual Aesthetic Elements to Consider*:  
  
1. **Paragraph Structure**: Ensure paragraphs are of appropriate length and logically structured.  
2. **Indentation**: Apply consistent indentation where necessary.  
3. **Headings and Subheadings**: Use headings and subheadings to organize content and improve readability.  
4. **Lists and Bullet Points**: Utilize lists and bullet points to break down complex information.  
5. **Markdown Formatting**: Apply markdown for bold, italic, and code blocks to highlight important information or distinguish between different types of text.  
6. **Line Spacing**: Adjust line spacing to enhance readability.  
7. **Consistency**: Maintain a consistent style throughout the document.  
8. **Visual Breaks**: Use visual breaks to separate different sections and improve flow.  
"""
user_template = \
"""### User Instruction:  
{instruction}  
  
### Answer:  
{completion}  
  
Your task is to:  
  
1. Analyze the LLM-generated response from a textual aesthetic perspective.  
2. Determine whether the text needs modification.  
3. Provide a revised version of the text if necessary.  
  
*Format*   
  
Textual Aesthetic Analysis:
[Your analysis]

Does it need modification:
[Y/N]

Revised Text:
[Your revised text, if applicable]
  
*Textual Aesthetic Elements to Consider*:  
  
1. **Paragraph Structure**: Ensure paragraphs are of appropriate length and logically structured.  
2. **Indentation**: Apply consistent indentation where necessary.  
3. **Headings and Subheadings**: Use headings and subheadings to organize content and improve readability.  
4. **Lists and Bullet Points**: Utilize lists and bullet points to break down complex information.  
5. **Markdown Formatting**: Apply markdown for bold, italic, and code blocks to highlight important information or distinguish between different types of text.  
6. **Line Spacing**: Adjust line spacing to enhance readability.  
7. **Consistency**: Maintain a consistent style throughout the document.  
8. **Visual Breaks**: Use visual breaks to separate different sections and improve flow.  
"""


In [32]:
from gpt4o import Openai, API_INFOS
client = Openai(apis=API_INFOS)
# ans = client.get_response("你是gpt几？", system="you are gpt2.5, a helpful chat assisstant")
# print(ans)
def get_revised_text(client, instruction, completion, user_template, system_template, max_tokens=2048):
    content = user_template.format(instruction=instruction, completion=completion)
    gpt_answer = client.get_response(content=content, system=system_template, max_tokens=max_tokens)
    gpt_answer = gpt_answer.strip()
    need_modification = "Y" if "Does it need modification: Y" in gpt_answer else "N"  
    revised_text_start = gpt_answer.find("Revised Text:") + len("Revised Text:")  
    revised_text = gpt_answer[revised_text_start:].strip() 
    return need_modification, revised_text, gpt_answer  

In [ ]:
# print(ans)
def get_revised_text(client, instruction, completion, user_template, system_template, max_tokens=2048):
    content = user_template.format(instruction=instruction, completion=completion)
    gpt_answer = client.get_response(content=content, system=system_template, max_tokens=max_tokens)
    gpt_answer = gpt_answer.strip()
    need_modification = "Y" if "Does it need modification: Y" in gpt_answer else "N"  
    revised_text_start = gpt_answer.find("Revised Text:") + len("Revised Text:")  
    revised_text = gpt_answer[revised_text_start:].strip() 
    return need_modification, revised_text, gpt_answer  
from tqdm import tqdm
revised_data = []  
sample_data = export_data.select(range(100))
for row in tqdm(sample_data):
    prompt = row['prompt']  
    response = row['response']

    need_modification, revised_text, gpt_answer = get_revised_text(client, prompt, response, user_template, system_template, max_tokens=2048) 
    # 将结果添加到列表  
    revised_data.append({  
        'prompt': prompt,  
        'response': response,  
        'does_it_need_modification': need_modification,  
        'revised_text': revised_text,
        'gpt_answer': gpt_answer 
    })  
# 创建新的Dataset  
revised_dataset = Dataset.from_pandas(pd.DataFrame(revised_data))  

In [36]:
from tqdm import tqdm
revised_data = []  
sample_data = export_data.select(range(100))
for row in tqdm(sample_data):
    prompt = row['prompt']  
    response = row['response']

    need_modification, revised_text, gpt_answer = get_revised_text(client, prompt, response, user_template, system_template, max_tokens=2048) 
    # 将结果添加到列表  
    revised_data.append({  
        'prompt': prompt,  
        'response': response,  
        'does_it_need_modification': need_modification,  
        'revised_text': revised_text,
        'gpt_answer': gpt_answer 
    })  
# 创建新的Dataset  
revised_dataset = Dataset.from_pandas(pd.DataFrame(revised_data))  

 13%|█▎        | 13/100 [02:35<17:19, 11.94s/it]

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

 14%|█▍        | 14/100 [02:49<18:03, 12.60s/it]

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}


 38%|███▊      | 38/100 [07:36<08:12,  7.95s/it]

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

 39%|███▉      | 39/100 [08:05<14:16, 14.05s/it]

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


 95%|█████████▌| 95/100 [17:58<01:07, 13.50s/it]

In [11]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed  
import os
import pandas as pd
from gpt4o import Openai, API_INFOS
def get_revised_text(client, instruction, completion, user_template, system_template, max_tokens=2048):  
    content = user_template.format(instruction=instruction, completion=completion)  
    # print(f"content: {content}")
    gpt_answer = client.get_response(content=content, system=system_template, max_tokens=max_tokens)  
    gpt_answer = gpt_answer.strip()  
    need_modification = "Y" if "Does it need modification: Y" in gpt_answer else "N"  
    revised_text_start = gpt_answer.find("Revised Text:") + len("Revised Text:")  
    revised_text = gpt_answer[revised_text_start:].strip()  
    return need_modification, revised_text, gpt_answer  
  
def process_row(index, client, row, user_template, system_template, max_tokens=2048, output_file="output.jsonl"):  
    prompt = row['prompt']  
    response = row['response']  
    need_modification, revised_text, gpt_answer = get_revised_text(client, prompt, response, user_template, system_template, max_tokens=max_tokens)  
    # print(f"index {index}")
    result = {  
        'index': index,  
        'prompt': prompt,  
        'response': response,  
        'does_it_need_modification': need_modification,  
        'revised_text': revised_text,  
        'gpt_answer': gpt_answer  
    }  
    # Write the result to a JSONL file  
    with open(output_file, 'a') as f:  
        f.write(json.dumps(result) + "\n")  
    return result  
def main():  
    # Initialize multiple clients  
    clients = [Openai(apis=[API_INFOS[i]]) for i in range(6)]  
  
    sample_data = export_data.select(range(100))
    # user_template = "User: {instruction}\nCompletion: {completion}"  
    # system_template = "You are a helpful assistant."  
    max_tokens = 2048  
    output_file = "./revised_data/output.jsonl"  
  
    # Clear the output file before starting  
    if os.path.exists(output_file):  
        os.remove(output_file)  
  
    revised_data = []  
  
    with ThreadPoolExecutor(max_workers=6) as executor:  
        # Create a future for each row in the dataset  
        futures = [executor.submit(process_row, i, clients[i % len(clients)], row, user_template, system_template, max_tokens, output_file) for i, row in enumerate(sample_data)]  
  
        # Collect the results as they complete  
        # for future in tqdm(as_completed(futures), total=len(futures)):  
        #     revised_data.append(future.result())  
        with tqdm(total=len(futures)) as pbar:  
            # Collect the results as they complete  
            for future in as_completed(futures):  
                result = future.result()  
                revised_data.append(result)  
                pbar.update(1)  # Update the progress bar
  
    # Load results from JSONL file and ensure the order is preserved  
    with open(output_file, 'r') as f:  
        revised_data = [json.loads(line) for line in f]  
  
    # Sort by the original index  
    revised_dataset = revised_data.sort(key=lambda x: x['index'])  
  
    # Create a new Dataset  
    revised_dataset = Dataset.from_pandas(pd.DataFrame(revised_data))  
    revised_dataset.to_json("./revised_data/output_sorted.jsonl") 
main()

KeyboardInterrupt: 

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

In [34]:
data_length = revised_dataset.shape[0]
revised_dataset.to_json(f"./revised_data/revised_{data_length}.jsonl")
revised_dataset

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 334.07ba/s]


Dataset({
    features: ['prompt', 'response', 'does_it_need_modification', 'revised_text', 'gpt_answer'],
    num_rows: 10
})

In [27]:
print(revised_data[1]['gpt_answer'])

Textual Aesthetic Analysis: The LLM-generated response is clear but lacks structural elements that could enhance readability and organization. The response could benefit from the use of headings, bullet points, and possibly some markdown formatting to emphasize key points. Additionally, the response could be broken down into more digestible parts.

Does it need modification: Y

Revised Text:

---

### Transforming the `getPosition` Method of `antv/g`'s Group in ZRender

It is generally not recommended to modify built-in methods as it can lead to unexpected results and potential bugs. Here are some alternative approaches you can consider:

1. **Develop a New Method**: 
   - Instead of modifying the existing method, create a new method that achieves your desired outcome.

2. **Explore Other Methods**:
   - Look into other methods provided by the library that might offer the functionality you need.

3. **Search for Other Libraries or Modules**:
   - There might be other libraries or modul

In [30]:
revised_data[3]

{'prompt': 'Which animal has two hands, a hyrax or a dog?',
 'response': 'Neither a hyrax nor a dog has hands. Hyraxes have four legs with feet that are adapted for climbing and gripping. Similarly, dogs also have four legs with paws. The term "hands" is usually reserved for primates, such as humans, which have opposable thumbs and fingers.',
 'does_it_need_modification': 'Y',
 'revised_text': '---\n\n### Which Animal Has Two Hands, a Hyrax or a Dog?\n\nNeither a hyrax nor a dog has hands. Here’s a breakdown of their limb structures:\n\n#### Hyrax\n- **Legs**: Four legs\n- **Feet**: Adapted for climbing and gripping\n\n#### Dog\n- **Legs**: Four legs\n- **Paws**: Adapted for walking and running\n\nThe term "hands" is usually reserved for primates, such as humans, which have opposable thumbs and fingers.\n\n---\n\nThis revised version uses headings and bullet points to organize the information, making it more readable and visually appealing.',
 'gpt_answer': 'Textual Aesthetic Analysis:

In [29]:
print(revised_data[3]['gpt_answer'])

Textual Aesthetic Analysis: The LLM-generated response is clear and accurate but could benefit from improved structure and formatting to enhance readability. The information can be organized into smaller sections with headings and bullet points to make it more visually appealing and easier to digest.

Does it need modification: Y

Revised Text:

---

### Which Animal Has Two Hands, a Hyrax or a Dog?

Neither a hyrax nor a dog has hands. Here’s a breakdown of their limb structures:

#### Hyrax
- **Legs**: Four legs
- **Feet**: Adapted for climbing and gripping

#### Dog
- **Legs**: Four legs
- **Paws**: Adapted for walking and running

The term "hands" is usually reserved for primates, such as humans, which have opposable thumbs and fingers.

---

This revised version uses headings and bullet points to organize the information, making it more readable and visually appealing.


In [39]:
export_data.select(range(100))

Dataset({
    features: ['prompt', 'response'],
    num_rows: 100
})

In [40]:
export_data.select(range(100)).to_json("uf_sample.jsonl")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 250.24ba/s]


216011